# ETL para datos de gestión de pedidos en e-commerce

Presentado por: Samuel Escalante

---

In [1]:
import pandas as pd
import os
import uuid

In [2]:
def summary(df: pd.DataFrame, name="DataFrame"):
    df_summary = pd.DataFrame({
        'dtypes': df.dtypes,
        'null count': df.isnull().sum(),
        'unique values': df.nunique(),
        'duplicate count': df.apply(lambda col: col.duplicated().sum()),
    })
    print(f"\n=== Summary for {name} ===")
    print(df_summary)

In [ ]:
folder_path = "../data"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path, sep=';')
            summary(df, name=filename)
        except Exception as e:
            print(f"Error leyendo {filename}: {e}")


=== Summary for Fecom Inc Customer List.csv ===
                       dtypes  null count  unique values  duplicate count
Customer_Trx_ID        object        3286          99441             3285
Subscriber_ID          object           0          99382             3345
Subscribe_Date         object           0           1349           101378
First_Order_Date       object        3286            624           102102
Customer_Postal_Code   object           0           1288           101439
Customer_City          object           0            265           102462
Customer_Country       object           0             27           102700
Customer_Country_Code  object           0             27           102700
Age                     int64           0             57           102670
Gender                 object           0              2           102725


C:\Users\saale\AppData\Local\Temp\ipykernel_15624\685232099.py:9: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';')



=== Summary for Fecom Inc Geolocations.csv ===
                  dtypes  null count  unique values  duplicate count
Geo_Postal_Code   object      998478           1669           998493
Geo_Lat           object      998478            333           999829
Geo_Lon           object      998478            335           999827
Geolocation_City  object      998478            337           999825
Geo_Country       object      998478             27          1000135

=== Summary for Fecom Inc Order Items.csv ===
                      dtypes  null count  unique values  duplicate count
Order_ID              object           0          98666            13984
Order_Item_ID          int64           0             21           112629
Product_ID            object           0          32951            79699
Seller_ID             object           0           3095           109555
Shipping_Limit_Date   object           0          54615            58035
Price                float64           0           59

---

# EDA

### Customers

In [4]:
customer = pd.read_csv("../data/Fecom Inc Customer List.csv", sep=';')
print(f"Shape of customer DataFrame: \n - columnas:{customer.shape[1]} \n - filas:{customer.shape[0]}")
customer.head(5)

Shape of customer DataFrame: 
 - columnas:10 
 - filas:102727


,Customer_Trx_ID,Subscriber_ID,Subscribe_Date,First_Order_Date,Customer_Postal_Code,Customer_City,Customer_Country,Customer_Country_Code,Age,Gender
0,1e959e1f5920cba43823fa9f95673b83,9765e039028279fd2e60bb620a451526,2023-07-08,2023-07-09,FR-75005,Paris,France,FR,29,Male
1,9877437582f263da7d7e30a90c57b8bb,a75e134e7eb6f96e2b0c716ac2a82efb,2024-03-23,2024-04-11,PL-00-001,Warsaw,Poland,PL,38,Male
2,fa6fbbb2080646acae977bf2e44af98b,2fdac27295500e820e43910e9a0aa8d8,2023-05-12,2023-06-01,NL-1012,Amsterdam,Netherlands,NL,35,Female
3,a4c9ff14ae7620126461ca55f36a76ea,e9ab8fd8ea96c85be2714c7f573fb7cd,2023-04-16,2023-04-26,IT-00144,Rome,Italy,IT,62,Male
4,93d5e378ae2f72a07db704f1f6716a7e,6384e6a7b021717616f25625b4bb0bf9,2023-05-26,2023-06-27,NL-1011,Amsterdam,Netherlands,NL,19,Male


In [5]:
customer.isnull().sum()

Customer_Trx_ID          3286
Subscriber_ID               0
Subscribe_Date              0
First_Order_Date         3286
Customer_Postal_Code        0
Customer_City               0
Customer_Country            0
Customer_Country_Code       0
Age                         0
Gender                      0
dtype: int64

In [6]:
customer_id_null = customer[customer['Customer_Trx_ID'].isnull()]
print(f"Customer IDs with null values: {customer_id_null}")

Customer IDs with null values:        Customer_Trx_ID                     Subscriber_ID Subscribe_Date  \
99441              NaN  0e0c08e088ec486e8784aefba35e17bc     2022-11-13   
99442              NaN  3ef786f65f114a27880ecdde737c638f     2023-06-12   
99443              NaN  2999799cc1a846b08098320f5aa3d8ad     2023-02-01   
99444              NaN  ed86c09317f34e209d497d04e3f84504     2024-08-22   
99445              NaN  a76544aba7844c7db3b1ab9bdcd7954e     2023-10-22   
...                ...                               ...            ...   
102722             NaN  fb53848e076b4863a0e3ce0750bde66e     2024-09-14   
102723             NaN  40ed38022bce40ad9f79dd1b53a2499b     2023-03-27   
102724             NaN  ef33bf0a9b1742a3a0944fb603e81c06     2024-09-17   
102725             NaN  abb296dcaa8f4961a6ea6681eff35e2c     2024-09-02   
102726             NaN  c2c686d021ed446cb0bb13552b85052f     2023-11-17   

       First_Order_Date Customer_Postal_Code Customer_City Customer_

In [7]:
# Conjunto de IDs ya usados (para no repetir)
existing_ids = set(customer['Customer_Trx_ID'].dropna())

# Función para generar un ID único
def generate_unique_id(existing_ids):
    while True:
        new_id = uuid.uuid4().hex  # genera un id hexadecimal único
        if new_id not in existing_ids:
            existing_ids.add(new_id)
            return new_id

# Reemplazar NaN con nuevos IDs
customer['Customer_Trx_ID'] = customer['Customer_Trx_ID'].apply(
    lambda x: generate_unique_id(existing_ids) if pd.isna(x) or x.lower() == 'nan' else x
)

In [70]:
customer[customer['Customer_Trx_ID'] == '9ef432eb6251297304e76186b10a928d']

,Customer_Trx_ID,Subscriber_ID,Subscribe_Date,First_Order_Date,Customer_Postal_Code,Customer_City,Customer_Country,Customer_Country_Code,Age,Gender


In [8]:
customer.isnull().sum()


Customer_Trx_ID             0
Subscriber_ID               0
Subscribe_Date              0
First_Order_Date         3286
Customer_Postal_Code        0
Customer_City               0
Customer_Country            0
Customer_Country_Code       0
Age                         0
Gender                      0
dtype: int64

In [9]:
duplicates = customer[customer.duplicated(subset=['Subscriber_ID'], keep=False)]
duplicates

,Customer_Trx_ID,Subscriber_ID,Subscribe_Date,First_Order_Date,Customer_Postal_Code,Customer_City,Customer_Country,Customer_Country_Code,Age,Gender
13,dc11c734af06298550995bb5ff673dce,a7f45c1961dbd98e7a14f3d38ea11c06,2024-05-20,2024-07-18,DE-50674,Cologne,Germany,DE,35,Female
19,02625456293ab29f0b11a84835a8c0ab,570eb70ff97166b85ea96be3bfb65fef,2023-10-10,2023-10-24,NL-1015,Amsterdam,Netherlands,NL,50,Male
28,6a6c433159ff775ba7f5ff8ec38b5b6f,b0bc6de4a99007470bbdbbf5cc502ea9,2022-11-09,2023-01-14,DE-10117,Berlin,Germany,DE,53,Male
40,0e24c20739fc582fda3306fc43683aa1,8cfc5350efe3bce6a0d6f96fbaf2c3b4,2021-10-03,2023-05-14,NL-2514,The Hague (Den Haag),Netherlands,NL,20,Female
59,4280fa78d7c43abc5e98a8a3b44d9350,4cfa5155cf7cff8eb15e0b12041d058e,2023-07-22,2023-08-04,DE-80331,Munich,Germany,DE,55,Female
...,...,...,...,...,...,...,...,...,...,...
99295,38a207eb1095e45ba5d6a3fae5112be2,1af3dcbc89998031651925e3994eab18,2023-05-11,2023-06-25,DE-10117,Berlin,Germany,DE,25,Male
99305,149ac0fdc3fc6ed91440c95c1a92736b,f1bee02ceae8a430506b798786f766b4,2024-03-25,2024-03-27,PT-1067,Lisbon,Portugal,PT,37,Male
99351,3059fc651e9f8e3a5701575e62f1ed1e,6f302f15a62c56b3100b67a81bfef328,2023-07-11,2023-07-18,DE-10115,Berlin,Germany,DE,34,Female
99391,aba4b03424668feae599fff4122142c8,16c8e42089546fde55ed3caa46478e7a,2023-10-23,2023-11-11,SE-40020,Gothenburg,Sweden,SE,48,Male


In [10]:
filtered_duplicates = duplicates[duplicates['Subscriber_ID'] == '570eb70ff97166b85ea96be3bfb65fef']
filtered_duplicates

,Customer_Trx_ID,Subscriber_ID,Subscribe_Date,First_Order_Date,Customer_Postal_Code,Customer_City,Customer_Country,Customer_Country_Code,Age,Gender
19,02625456293ab29f0b11a84835a8c0ab,570eb70ff97166b85ea96be3bfb65fef,2023-10-10,2023-10-24,NL-1015,Amsterdam,Netherlands,NL,50,Male
56751,00796be07bfc87dad2e31ef15490a718,570eb70ff97166b85ea96be3bfb65fef,2023-10-10,2023-10-24,NL-1011,Amsterdam,Netherlands,NL,50,Male


In [11]:
customer = customer.drop_duplicates(subset=['Subscriber_ID'], keep='first')

In [12]:
duplicates = customer[customer.duplicated(subset=['Subscriber_ID'], keep=False)]
duplicates

,Customer_Trx_ID,Subscriber_ID,Subscribe_Date,First_Order_Date,Customer_Postal_Code,Customer_City,Customer_Country,Customer_Country_Code,Age,Gender


In [13]:
customer.isnull().sum()

Customer_Trx_ID             0
Subscriber_ID               0
Subscribe_Date              0
First_Order_Date         3286
Customer_Postal_Code        0
Customer_City               0
Customer_Country            0
Customer_Country_Code       0
Age                         0
Gender                      0
dtype: int64

In [14]:
same_date = customer[customer['First_Order_Date'] == customer['Subscribe_Date']]
same_date

,Customer_Trx_ID,Subscriber_ID,Subscribe_Date,First_Order_Date,Customer_Postal_Code,Customer_City,Customer_Country,Customer_Country_Code,Age,Gender
18,ff24c0250b3f882f90d87942fb7583a9,5950105d5f3a678d89a7e0d2a54b5eaa,2023-06-27,2023-06-27,FR-83400,Toulon,France,FR,25,Male
65,fc4864717c28d7c6f7c77ba6fffca526,f43a0abc09ca9be20e26630515a8a623,2024-03-17,2024-03-17,AT-1050,Vienna,Austria,AT,30,Male
82,eb9a02ebb55bcfc0516d56555290bfa4,f6c7e1a95cb61fd044a33c6a38f2dad8,2023-02-27,2023-02-27,NL-7313,Apeldoorn,Netherlands,NL,36,Male
98,accdad8ab1a62db957ffcefa272667a0,339f4ca769f8240ea30f88c25c5c5f11,2023-11-16,2023-11-16,GB-W1,London,United Kingdom,GB,32,Male
120,bbf249561e84f966c88f8d70e9c714fc,76888ec0e2d5b7de57da85ce2babccac,2023-08-21,2023-08-21,DE-10117,Berlin,Germany,DE,38,Male
...,...,...,...,...,...,...,...,...,...,...
99253,c8db401ec63a5cf4fcc9e331cc626c96,f120fc6d61289eefb3f39aac12abb022,2024-07-28,2024-07-28,DE-33607,Bielefeld,Germany,DE,59,Male
99304,0f3a6d2c22bbdb61d02eb98f4a1e0dfb,c5cc879614b6acdf5b62db3edd062e81,2023-12-30,2023-12-30,FR-75003,Paris,France,FR,26,Female
99347,3610566c1bd35ade63452acf27133f22,51ff8247356916c4e62abd8ec002e0c9,2023-02-20,2023-02-20,DE-20146,Hamburg,Germany,DE,35,Male
99367,bb5ecb070b1cabb71470e74ccabb9ad5,f92f6ab3c62a0ea894141b1f06b3f160,2024-02-11,2024-02-11,DK-6730,Esbjerg,Denmark,DK,18,Female


In [15]:
customer['First_Order_Date'] = customer['First_Order_Date'].fillna(customer['Subscribe_Date'])


In [16]:
customer.isnull().sum()

Customer_Trx_ID          0
Subscriber_ID            0
Subscribe_Date           0
First_Order_Date         0
Customer_Postal_Code     0
Customer_City            0
Customer_Country         0
Customer_Country_Code    0
Age                      0
Gender                   0
dtype: int64

### Descubrimientos clave:
- El dataset cuenta con 2 columnas con valores faltantes: `Customer_Trx_ID` y `First_Order_Date`.

### Transformaciones aplicadas:
- **Imputación de IDs faltantes en `Customer_Trx_ID`:**  
  Se generaron identificadores únicos con `uuid.uuid4()` para aquellos registros donde el valor estaba nulo o era inválido (`NaN`). Esto asegura que cada cliente tenga un identificador único.
  
- **Eliminación de duplicados en `Subscriber_ID`:**  
  Se conservaron únicamente los registros únicos por `Subscriber_ID`, manteniendo el primer registro y eliminando los duplicados posteriores.
  
- **Imputación de fechas faltantes en `First_Order_Date`:**  
  Cuando `First_Order_Date` estaba vacío, se reemplazó por la fecha de `Subscribe_Date`, garantizando consistencia temporal en el historial del cliente.

---

## Geolocations

In [17]:
geolocations = pd.read_csv("../data/Fecom Inc Geolocations.csv", sep=';')
print(f"Shape of geolocations DataFrame: \n - columnas:{geolocations.shape[1]} \n - filas:{geolocations.shape[0]}")

Shape of geolocations DataFrame: 
 - columnas:5 
 - filas:1000163


C:\Users\saale\AppData\Local\Temp\ipykernel_15624\3515723160.py:1: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  geolocations = pd.read_csv("../data/Fecom Inc Geolocations.csv", sep=';')


In [18]:
geolocations.head(5)

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country
0,NL-5211,"51,7000","5,3167",'s-Hertogenbosch,Netherlands
1,NL-5212,"51,7000","5,3167",'s-Hertogenbosch,Netherlands
2,NL-5213,"51,7000","5,3167",'s-Hertogenbosch,Netherlands
3,NL-5214,"51,7000","5,3167",'s-Hertogenbosch,Netherlands
4,NL-5215,"51,7000","5,3167",'s-Hertogenbosch,Netherlands


In [19]:
geolocations.isnull().sum()

Geo_Postal_Code     998478
Geo_Lat             998478
Geo_Lon             998478
Geolocation_City    998478
Geo_Country         998478
dtype: int64

In [20]:
geolocations_null = geolocations[geolocations.isnull()]
print(f"Customer IDs with null values: {geolocations_null}")

Customer IDs with null values:         Geo_Postal_Code Geo_Lat Geo_Lon Geolocation_City Geo_Country
0                   NaN     NaN     NaN              NaN         NaN
1                   NaN     NaN     NaN              NaN         NaN
2                   NaN     NaN     NaN              NaN         NaN
3                   NaN     NaN     NaN              NaN         NaN
4                   NaN     NaN     NaN              NaN         NaN
...                 ...     ...     ...              ...         ...
1000158             NaN     NaN     NaN              NaN         NaN
1000159             NaN     NaN     NaN              NaN         NaN
1000160             NaN     NaN     NaN              NaN         NaN
1000161             NaN     NaN     NaN              NaN         NaN
1000162             NaN     NaN     NaN              NaN         NaN

[1000163 rows x 5 columns]


In [21]:
geolocations = geolocations.dropna()
geolocations.isnull().sum()

Geo_Postal_Code     0
Geo_Lat             0
Geo_Lon             0
Geolocation_City    0
Geo_Country         0
dtype: int64

In [22]:
geolocations.shape

(1685, 5)

In [23]:
geolocations_duplicates = geolocations[geolocations.duplicated(subset=['Geo_Postal_Code'], keep=False)]
geolocations_duplicates

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country
40,PT-2800,"38,6760","-9,1650",Almada,Portugal
44,PT-2840,"38,6760","-9,1650",Almada,Portugal
51,PT-2705,"38,7500","-9,2333",Amadora,Portugal
52,PT-2710,"38,7500","-9,2333",Amadora,Portugal
247,BE-1030,"50,8467","4,3525",Brussels,Belgium
249,BE-1050,"50,8467","4,3525",Brussels,Belgium
269,PT-2785,"38,6970","-9,4215",Cascais,Portugal
451,PT-4430,"41,1440","-8,5322",Gondomar,Portugal
453,PT-4450,"41,1440","-8,5322",Gondomar,Portugal
454,PT-4460,"41,1440","-8,5322",Gondomar,Portugal


In [24]:
geolocations_duplicates.shape

(32, 5)

In [25]:
geolocations_duplicates_filtered = geolocations_duplicates[geolocations_duplicates['Geo_Postal_Code'] == 'AT-4050']
geolocations_duplicates_filtered

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country
693,AT-4050,"48,3058","14,2864",Linz,Austria
1640,AT-4050,"48,2217","14,2397",Traun,Austria


In [26]:
geolocations[geolocations['Geo_Postal_Code'] == 'FR-75005']

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country
959,FR-75005,"48,8567","2,3522",Paris,France


In [27]:
# Si las columnas tienen comas en lugar de puntos (en lat/lon), corrígelo antes (opcional)
geolocations['Geo_Lat'] = geolocations['Geo_Lat'].astype(str).str.replace(',', '.').astype(float)
geolocations['Geo_Lon'] = geolocations['Geo_Lon'].astype(str).str.replace(',', '.').astype(float)

# Aplicar el código postal representativo a cada fila según la ciudad
postal_by_city = geolocations.groupby("Geolocation_City")["Geo_Postal_Code"].agg(lambda x: x.mode().iloc[0])
geolocations["Geo_Postal_Code_Mode"] = geolocations["Geolocation_City"].map(postal_by_city)

# (Opcional) Eliminar duplicados exactos ahora que los códigos son únicos por ciudad
geolocations = geolocations.drop_duplicates()

In [28]:
geolocations[geolocations['Geo_Postal_Code'] == 'FR-75005']

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country,Geo_Postal_Code_Mode
959,FR-75005,48.8567,2.3522,Paris,France,FR-75001


In [29]:
geolocations_duplicates = geolocations[geolocations.duplicated(subset=['Geo_Postal_Code'], keep=False)]
geolocations_duplicates

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country,Geo_Postal_Code_Mode
40,PT-2800,38.6760,-9.1650,Almada,Portugal,PT-2800
44,PT-2840,38.6760,-9.1650,Almada,Portugal,PT-2800
51,PT-2705,38.7500,-9.2333,Amadora,Portugal,PT-2700
52,PT-2710,38.7500,-9.2333,Amadora,Portugal,PT-2700
247,BE-1030,50.8467,4.3525,Brussels,Belgium,BE-1000
249,BE-1050,50.8467,4.3525,Brussels,Belgium,BE-1000
269,PT-2785,38.6970,-9.4215,Cascais,Portugal,PT-2750
451,PT-4430,41.1440,-8.5322,Gondomar,Portugal,PT-4420
453,PT-4450,41.1440,-8.5322,Gondomar,Portugal,PT-4420
454,PT-4460,41.1440,-8.5322,Gondomar,Portugal,PT-4420


In [30]:
geolocations.isnull().sum()

Geo_Postal_Code         0
Geo_Lat                 0
Geo_Lon                 0
Geolocation_City        0
Geo_Country             0
Geo_Postal_Code_Mode    0
dtype: int64

In [31]:
geolocations

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country,Geo_Postal_Code_Mode
0,NL-5211,51.7000,5.3167,'s-Hertogenbosch,Netherlands,NL-5211
1,NL-5212,51.7000,5.3167,'s-Hertogenbosch,Netherlands,NL-5211
2,NL-5213,51.7000,5.3167,'s-Hertogenbosch,Netherlands,NL-5211
3,NL-5214,51.7000,5.3167,'s-Hertogenbosch,Netherlands,NL-5211
4,NL-5215,51.7000,5.3167,'s-Hertogenbosch,Netherlands,NL-5211
...,...,...,...,...,...,...
1680,NL-8011,52.5167,6.1000,Zwolle,Netherlands,NL-8011
1681,NL-8012,52.5167,6.1000,Zwolle,Netherlands,NL-8011
1682,NL-8013,52.5167,6.1000,Zwolle,Netherlands,NL-8011
1683,NL-8014,52.5167,6.1000,Zwolle,Netherlands,NL-8011


In [32]:
geolocations[geolocations['Geo_Postal_Code'] == 'FR-75001']

,Geo_Postal_Code,Geo_Lat,Geo_Lon,Geolocation_City,Geo_Country,Geo_Postal_Code_Mode
955,FR-75001,48.8567,2.3522,Paris,France,FR-75001


### Descubrimientos clave:
- El dataset contenía valores faltantes en varias filas.  
- Las coordenadas (`Geo_Lat` y `Geo_Lon`) tenían algunos registros con comas en lugar de puntos decimales.  
- Existían múltiples códigos postales por ciudad, generando inconsistencia en los datos.  

### Transformaciones aplicadas:
- **Eliminación de valores faltantes:**  
  Se removieron todas las filas que contenían valores nulos.  

- **Corrección de formato en coordenadas:**  
  Se reemplazaron las comas por puntos en las columnas `Geo_Lat` y `Geo_Lon`, y luego se transformaron a tipo numérico (`float`).  

- **Imputación de códigos postales por ciudad:**  
  Se calculó el **código postal representativo** por ciudad utilizando la moda de `Geo_Postal_Code`.  
  Posteriormente, se asignó este valor a cada registro en la columna `Geo_Postal_Code_Mode`.  

- **Eliminación de duplicados:**  
  Una vez corregidos los códigos postales, se eliminaron las filas duplicadas para garantizar que cada combinación de ubicación sea única.  

---

## Orders Item

In [33]:
orders_items = pd.read_csv("../data/Fecom Inc Order Items.csv", sep=';')
orders_items.shape

(112650, 7)

In [34]:
orders_items.head(5)

,Order_ID,Order_Item_ID,Product_ID,Seller_ID,Shipping_Limit_Date,Price,Freight_Value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2023-09-19 09:45,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2023-05-03 11:05,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2024-01-18 14:48,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2024-08-15 10:10,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2023-02-13 13:57,199.90,18.14


In [35]:
orders_items.isnull().sum()

Order_ID               0
Order_Item_ID          0
Product_ID             0
Seller_ID              0
Shipping_Limit_Date    0
Price                  0
Freight_Value          0
dtype: int64

In [36]:
orders_items_duplicates = orders_items[orders_items.duplicated(subset=['Order_ID'], keep=False)]
orders_items_duplicates

,Order_ID,Order_Item_ID,Product_ID,Seller_ID,Shipping_Limit_Date,Price,Freight_Value
13,0008288aa423d2a3f00fcb17cd7d8719,1,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2024-02-21 02:55,49.90,13.37
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2024-02-21 02:55,49.90,13.37
32,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2023-10-20 16:07,21.33,15.10
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2023-10-20 16:07,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2023-10-20 16:07,21.33,15.10
...,...,...,...,...,...,...,...
112635,fff8287bbae429a99bb7e8c21d151c41,2,bee2e070c39f3dd2f6883a17a5f0da45,4e922959ae960d389249c378d1c939f5,2024-03-27 12:29,180.00,48.14
112640,fffb9224b6fc7c43ebb0904318b10b5f,1,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,2023-11-03 02:55,55.00,34.19
112641,fffb9224b6fc7c43ebb0904318b10b5f,2,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,2023-11-03 02:55,55.00,34.19
112642,fffb9224b6fc7c43ebb0904318b10b5f,3,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,2023-11-03 02:55,55.00,34.19


### Descubrimientos clave:
- El dataset `orders_items` no presentó valores faltantes, inconsistencias ni duplicados relevantes.  
- La estructura y calidad de los datos fue adecuada desde el inicio.  

### Transformaciones aplicadas:
- No se realizaron transformaciones, ya que el dataset se encontraba **limpio y consistente**.  

---

## Orders Payments

In [37]:
orders_payments = pd.read_csv("../data/Fecom Inc Order Payments.csv", sep=';')
orders_payments.head(5)

,Order_ID,Payment_Sequential,Payment_Type,Payment_Installments,Payment_Value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [38]:
orders_payments.shape

(103886, 5)

In [39]:
orders_payments.isnull().sum()

Order_ID                0
Payment_Sequential      0
Payment_Type            0
Payment_Installments    0
Payment_Value           0
dtype: int64

In [40]:
orders_payments_duplicates = orders_payments[orders_payments.duplicated(subset=['Order_ID'], keep=False)]
orders_payments_duplicates

,Order_ID,Payment_Sequential,Payment_Type,Payment_Installments,Payment_Value
25,5cfd514482e22bc992e7693f0e3e8df7,2,voucher,1,45.17
35,b2bb080b6bc860118a246fd9b6fad6da,1,credit_card,1,173.84
75,3689194c14ad4e2e7361ebd1df0e77b0,2,voucher,1,57.53
84,723e462ce1ee50e024887c0b403130f3,1,credit_card,1,13.80
102,21b8b46679ea6482cbf911d960490048,2,voucher,1,43.12
...,...,...,...,...,...
103778,fd86c80924b4be8fb7f58c4ecc680dae,1,credit_card,1,76.10
103817,6d4616de4341417e17978fe57aec1c46,1,credit_card,1,19.18
103860,31bc09fdbd701a7a4f9b55b5955b8687,6,voucher,1,77.99
103869,c9b01bef18eb84888f0fd071b8413b38,1,credit_card,6,238.16


In [41]:
# Filtrar
orders_payments_filtered = orders_payments[
    (orders_payments['Order_ID'].duplicated(keep=False) & 
     orders_payments.groupby('Order_ID')['Payment_Sequential']
       .transform('nunique')
       .gt(1))  
    |
    (~orders_payments['Order_ID'].duplicated(keep=False))  
]

orders_payments_filtered

,Order_ID,Payment_Sequential,Payment_Type,Payment_Installments,Payment_Value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,debit_card,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


### Descubrimientos clave:
- El dataset `orders_payments` no presentó valores faltantes, inconsistencias ni duplicados relevantes.  
- Los registros ya tenían una buena consistencia en su estructura y calidad.  

### Transformaciones aplicadas:
- No se realizaron transformaciones, ya que el dataset se encontraba **limpio y consistente**.  


---

## Orders

In [42]:
orders = pd.read_csv("../data/Fecom Inc Orders.csv", sep=';')
orders.head(5)

,Order_ID,Customer_Trx_ID,Order_Status,Order_Purchase_Timestamp,Order_Approved_At,Order_Delivered_Carrier_Date,Order_Delivered_Customer_Date,Order_Estimated_Delivery_Date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2023-10-02 10:56,2023-10-02 11:07,2023-10-04 19:55,2023-10-10 21:25,2023-10-18 00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2024-07-24 20:41,2024-07-26 03:24,2024-07-26 14:31,2024-08-07 15:27,2024-08-13 00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2024-08-08 08:38,2024-08-08 08:55,2024-08-08 13:50,2024-08-17 18:06,2024-09-04 00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2023-11-18 19:28,2023-11-18 19:45,2023-11-22 13:39,2023-12-02 00:28,2023-12-15 00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2024-02-13 21:18,2024-02-13 22:20,2024-02-14 19:46,2024-02-16 18:17,2024-02-26 00:00


In [43]:
orders.isnull().sum()

Order_ID                            0
Customer_Trx_ID                     0
Order_Status                        0
Order_Purchase_Timestamp            0
Order_Approved_At                 160
Order_Delivered_Carrier_Date     1783
Order_Delivered_Customer_Date    2965
Order_Estimated_Delivery_Date       0
dtype: int64

In [44]:
orders_duplicates = orders[orders.duplicated(subset=['Order_ID'], keep=False)]
orders_duplicates

,Order_ID,Customer_Trx_ID,Order_Status,Order_Purchase_Timestamp,Order_Approved_At,Order_Delivered_Carrier_Date,Order_Delivered_Customer_Date,Order_Estimated_Delivery_Date


In [45]:
orders_date_null = orders[orders['Order_Approved_At'].isnull()]
orders_date_null

,Order_ID,Customer_Trx_ID,Order_Status,Order_Purchase_Timestamp,Order_Approved_At,Order_Delivered_Carrier_Date,Order_Delivered_Customer_Date,Order_Estimated_Delivery_Date
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2024-08-28 15:26,NaN,NaN,NaN,2024-09-12 00:00
1801,ed3efbd3a87bea76c2812c66a0b32219,191984a8ba4cbb2145acb4fe35b69664,canceled,2024-09-20 13:54,NaN,NaN,NaN,2024-10-17 00:00
1868,df8282afe61008dc26c6c31011474d02,aa797b187b5466bc6925aaaa4bb3bed1,canceled,2023-03-04 12:14,NaN,NaN,NaN,2023-04-10 00:00
2029,8d4c637f1accf7a88a4555f02741e606,b1dd715db389a2077f43174e7a675d07,canceled,2024-08-29 16:27,NaN,NaN,NaN,2024-09-13 00:00
2161,7a9d4c7f9b068337875b95465330f2fc,7f71ae48074c0cfec9195f88fcbfac55,canceled,2023-05-01 16:12,NaN,NaN,NaN,2023-05-30 00:00
...,...,...,...,...,...,...,...,...
97696,5a00b4d35edffc56b825c3646a99ba9d,6a3bdf004ca96338fb5fad1b8d93c2e6,canceled,2023-07-02 15:38,NaN,NaN,NaN,2023-07-25 00:00
98415,227c804e2a44760671a6a5697ea549e4,62e7477e75e542243ee62a0ba73f410f,canceled,2023-09-28 15:02,NaN,NaN,NaN,2023-10-16 00:00
98909,e49e7ce1471b4693482d40c2bd3ad196,e4e7ab3f449aeb401f0216f86c2104db,canceled,2024-08-07 11:16,NaN,NaN,NaN,2024-08-10 00:00
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2024-08-31 16:13,NaN,NaN,NaN,2024-10-01 00:00


missing not at random

In [46]:
orders_canceled = orders[orders['Order_Status'] == 'canceled']
orders_canceled

,Order_ID,Customer_Trx_ID,Order_Status,Order_Purchase_Timestamp,Order_Approved_At,Order_Delivered_Carrier_Date,Order_Delivered_Customer_Date,Order_Estimated_Delivery_Date
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2024-08-04 14:29,2024-08-07 04:10,NaN,NaN,2024-08-14 00:00
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2024-01-26 21:34,2024-01-26 21:58,2024-01-29 22:33,NaN,2024-02-22 00:00
1058,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2024-01-25 13:34,2024-01-25 13:50,2024-01-26 21:42,NaN,2024-02-23 00:00
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2024-08-28 15:26,NaN,NaN,NaN,2024-09-12 00:00
1801,ed3efbd3a87bea76c2812c66a0b32219,191984a8ba4cbb2145acb4fe35b69664,canceled,2024-09-20 13:54,NaN,NaN,NaN,2024-10-17 00:00
...,...,...,...,...,...,...,...,...
98791,b159d0ce7cd881052da94fa165617b05,e0c3bc5ce0836b975d6b2a8ce7bb0e3e,canceled,2023-03-11 19:51,2023-03-11 19:51,NaN,NaN,2023-03-30 00:00
98909,e49e7ce1471b4693482d40c2bd3ad196,e4e7ab3f449aeb401f0216f86c2104db,canceled,2024-08-07 11:16,NaN,NaN,NaN,2024-08-10 00:00
99143,6560fb10610771449cb0463c5ba12199,0d07d0a588caf93cc66b7a8aff86d2fe,canceled,2023-10-01 22:26,2023-10-01 22:35,NaN,NaN,2023-10-27 00:00
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2024-08-31 16:13,NaN,NaN,NaN,2024-10-01 00:00


In [47]:
# Definir la fecha bandera
date_flag = pd.Timestamp("1900-01-01")

# Reemplazar nulos en las tres columnas de fecha
orders = orders.fillna({
    'Order_Approved_At': date_flag,
    'Order_Delivered_Carrier_Date': date_flag,
    'Order_Delivered_Customer_Date': date_flag
})


In [48]:
orders_canceled = orders[orders['Order_Status'] == 'canceled']
orders_canceled

,Order_ID,Customer_Trx_ID,Order_Status,Order_Purchase_Timestamp,Order_Approved_At,Order_Delivered_Carrier_Date,Order_Delivered_Customer_Date,Order_Estimated_Delivery_Date
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2024-08-04 14:29,2024-08-07 04:10,1900-01-01 00:00:00,1900-01-01 00:00:00,2024-08-14 00:00
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2024-01-26 21:34,2024-01-26 21:58,2024-01-29 22:33,1900-01-01 00:00:00,2024-02-22 00:00
1058,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2024-01-25 13:34,2024-01-25 13:50,2024-01-26 21:42,1900-01-01 00:00:00,2024-02-23 00:00
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2024-08-28 15:26,1900-01-01 00:00:00,1900-01-01 00:00:00,1900-01-01 00:00:00,2024-09-12 00:00
1801,ed3efbd3a87bea76c2812c66a0b32219,191984a8ba4cbb2145acb4fe35b69664,canceled,2024-09-20 13:54,1900-01-01 00:00:00,1900-01-01 00:00:00,1900-01-01 00:00:00,2024-10-17 00:00
...,...,...,...,...,...,...,...,...
98791,b159d0ce7cd881052da94fa165617b05,e0c3bc5ce0836b975d6b2a8ce7bb0e3e,canceled,2023-03-11 19:51,2023-03-11 19:51,1900-01-01 00:00:00,1900-01-01 00:00:00,2023-03-30 00:00
98909,e49e7ce1471b4693482d40c2bd3ad196,e4e7ab3f449aeb401f0216f86c2104db,canceled,2024-08-07 11:16,1900-01-01 00:00:00,1900-01-01 00:00:00,1900-01-01 00:00:00,2024-08-10 00:00
99143,6560fb10610771449cb0463c5ba12199,0d07d0a588caf93cc66b7a8aff86d2fe,canceled,2023-10-01 22:26,2023-10-01 22:35,1900-01-01 00:00:00,1900-01-01 00:00:00,2023-10-27 00:00
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2024-08-31 16:13,1900-01-01 00:00:00,1900-01-01 00:00:00,1900-01-01 00:00:00,2024-10-01 00:00


In [49]:
orders.isnull().sum()

Order_ID                         0
Customer_Trx_ID                  0
Order_Status                     0
Order_Purchase_Timestamp         0
Order_Approved_At                0
Order_Delivered_Carrier_Date     0
Order_Delivered_Customer_Date    0
Order_Estimated_Delivery_Date    0
dtype: int64

In [71]:
orders[orders['Customer_Trx_ID'] == '9ef432eb6251297304e76186b10a928d']


,Order_ID,Customer_Trx_ID,Order_Status,Order_Purchase_Timestamp,Order_Approved_At,Order_Delivered_Carrier_Date,Order_Delivered_Customer_Date,Order_Estimated_Delivery_Date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2023-10-02 10:56,2023-10-02 11:07,2023-10-04 19:55,2023-10-10 21:25,2023-10-18 00:00


In [75]:
merged = orders.merge(customer, on="Customer_Trx_ID", how="left", indicator=True)
merged[merged["_merge"] == "left_only"]["Customer_Trx_ID"].unique()


array(['9ef432eb6251297304e76186b10a928d',
       '3b6828a50ffe546942b7a473d70ac0fc',
       'caded193e8e47b8362864762a83db3c5', ...,
       'e6f5f771cdbc5d09c994f16c9af6523d',
       '2f0524a7b1b3845a1a57fcf3910c4333',
       'ce5efd1b1d8a2fa290a871edd5c26cd6'], shape=(3345,), dtype=object)

### Descubrimientos clave:
- Se identificaron valores faltantes en las columnas de fechas:  
  - `Order_Approved_At`  
  - `Order_Delivered_Carrier_Date`  
  - `Order_Delivered_Customer_Date`  
- Existían registros con `Customer_Trx_ID` no presentes en la tabla de clientes (`customer`).  

### Transformaciones aplicadas:
- **Imputación de fechas faltantes:**  
  Los valores nulos en las columnas de fechas se reemplazaron con una fecha bandera `1900-01-01` para mantener la consistencia temporal en el dataset.  

- **Filtrado de registros por cliente válido:**  
  Se conservaron únicamente las órdenes cuyo `Customer_Trx_ID` se encuentra en la tabla `customer`, asegurando integridad referencial entre ambas tablas.  


---

## Producsts

In [50]:
products = pd.read_csv("../data/Fecom Inc Products.csv", sep=';')
products.head(5)

,Product_ID,Product_Category_Name,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm
0,1e9e8ef04dbcff4541ed26657ea517e5,Perfumery,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,Art,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,Sports_Leisure,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,Baby,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,Housewares,625.0,20.0,17.0,13.0


In [51]:
products.isnull().sum()

Product_ID                 0
Product_Category_Name    623
Product_Weight_Gr          2
Product_Length_Cm          2
Product_Height_Cm          2
Product_Width_Cm           2
dtype: int64

In [52]:
products_duplicates = products[products.duplicated(subset=['Product_ID'], keep=False)]
products_duplicates

,Product_ID,Product_Category_Name,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm


In [53]:
products_null = products[products.isnull().any(axis=1)]
products_null

,Product_ID,Product_Category_Name,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm
105,a41e356c76fab66334f36de622ecbd3a,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,NaN,1800.0,30.0,20.0,70.0
32589,10dbe0fbaa2c505123c17fdc34a63c56,NaN,800.0,30.0,10.0,23.0
32616,bd2ada37b58ae94cc838b9c0569fecd8,NaN,200.0,21.0,8.0,16.0
32772,fa51e914046aab32764c41356b9d4ea4,NaN,1300.0,45.0,16.0,45.0


In [54]:
products['Product_Category_Name'] = products['Product_Category_Name'].fillna('unknown')


In [55]:
products.isnull().sum()

Product_ID               0
Product_Category_Name    0
Product_Weight_Gr        2
Product_Length_Cm        2
Product_Height_Cm        2
Product_Width_Cm         2
dtype: int64

In [56]:
products_null = products[products.isnull().any(axis=1)]
products_null

,Product_ID,Product_Category_Name,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm
8578,09ff539a621711667c43eba6a3bd8466,Baby,NaN,NaN,NaN,NaN
18851,5eb564652db742ff8f28759cd8d2652a,unknown,NaN,NaN,NaN,NaN


In [57]:
for col in ['Product_Weight_Gr', 'Product_Length_Cm', 'Product_Height_Cm', 'Product_Width_Cm']:
    median_val = products[col].median()
    products[col] = products[col].fillna(median_val)


### Descubrimientos clave:
- La columna `Product_Category_Name` contenía valores faltantes.  
- Las columnas de medidas físicas del producto (`Product_Weight_Gr`, `Product_Length_Cm`, `Product_Height_Cm`, `Product_Width_Cm`) tenían valores nulos en algunos registros.  

### Transformaciones aplicadas:
- **Imputación en categorías de producto:**  
  Los valores faltantes en `Product_Category_Name` fueron reemplazados por la etiqueta `"unknown"`, evitando pérdida de registros y manteniendo la trazabilidad.  

- **Imputación en medidas físicas del producto:**  
  Los valores nulos en las columnas de peso y dimensiones (`Product_Weight_Gr`, `Product_Length_Cm`, `Product_Height_Cm`, `Product_Width_Cm`) se imputaron con la **mediana** de cada variable, garantizando consistencia sin sesgar los datos por valores extremos.  

---

## Sellers List

In [58]:
sellers = pd.read_csv("../data/Fecom Inc Sellers List.csv", sep=';')
sellers.head(5)

,Seller_ID,Seller_Name,Seller_Postal_Code,Seller_City,Country_Code,Seller_Country
0,d1b65fc7debc3361ea86b5f14c68d2e2,NeuroLabsX,DE-14469,Potsdam,DE,Germany
1,51a04a8a6bdcb23deccc82b0b80742cf,SwiftLabs,DE-6108,Halle (Saale),DE,Germany
2,e49c26c3edfa46d227d5121a6b6e4d37,EcoFutures,ES-33003,Oviedo,ES,Spain
3,1b938a7ec6ac5061a66a3766e0e75f90,HyperHub,DE-6112,Halle (Saale),DE,Germany
4,a7a9b880c49781da66651ccf4ba9ac38,EliteAI,DE-18069,Rostock,DE,Germany


In [59]:
sellers.isnull().sum()

Seller_ID             0
Seller_Name           0
Seller_Postal_Code    0
Seller_City           0
Country_Code          0
Seller_Country        0
dtype: int64

In [60]:
sellers_duplicates = sellers[sellers.duplicated(subset=['Seller_ID'], keep=False)]
sellers_duplicates

,Seller_ID,Seller_Name,Seller_Postal_Code,Seller_City,Country_Code,Seller_Country


### Descubrimientos clave:
- El dataset `sellers` no presentó valores faltantes, duplicados ni inconsistencias relevantes.  
- La calidad y estructura de los datos era adecuada desde el inicio.  

### Transformaciones aplicadas:
- No se realizaron transformaciones, ya que el dataset se encontraba **limpio y consistente**.  


---

## Orders Review

In [61]:
orders_reviews_no_emojis = pd.read_csv("../data/Fecom_Inc_Order_Reviews_No_Emojis.csv", sep=';')
orders_reviews_no_emojis.head(5)

,Review_ID,Order_ID,Review_Score,Review_Comment_Title_En,Review_Comment_Message_En,Review_Creation_Date,Review_Answer_Timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2024-01-18 00:00,2024-01-18 21:46
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2024-03-10 00:00,2024-03-11 03:05
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2024-02-17 00:00,2024-02-18 14:36
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated deadl...,2023-04-21 00:00,2023-04-21 22:02
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,"Congratulations lannister stores, I loved shop...",2024-03-01 00:00,2024-03-02 10:26


In [62]:
orders_reviews_no_emojis.isnull().sum()

Review_ID                        0
Order_ID                         0
Review_Score                     0
Review_Comment_Title_En      87682
Review_Comment_Message_En    58347
Review_Creation_Date             0
Review_Answer_Timestamp          0
dtype: int64

In [63]:
orders_reviews_no_emojis_no_null = orders_reviews_no_emojis[orders_reviews_no_emojis.notnull().all(axis=1)]
orders_reviews_no_emojis_no_null

,Review_ID,Order_ID,Review_Score,Review_Comment_Title_En,Review_Comment_Message_En,Review_Creation_Date,Review_Answer_Timestamp
9,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recommend,efficient device. On the website the brand of ...,2024-05-22 00:00,2024-05-23 16:45
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,I highly recommend,"Reliable seller, ok product and delivery on time.",2024-05-23 00:00,2024-05-24 03:00
19,373cbeecea8286a2b66c97b1b157ec46,583174fbe37d3d5f0d6661be3aad1786,1,My product didn't arrive,Terrible,2024-08-15 00:00,2024-08-15 04:10
22,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,Excellent,Store note 10,2024-07-10 00:00,2024-07-11 14:10
34,c92cdd7dd544a01aa35137f901669cdf,37e7875cdce5a9e5b3a692971f370151,4,Very good.,I received exactly what I expected. Other orde...,2024-06-07 00:00,2024-06-09 18:44
...,...,...,...,...,...,...,...
99062,adef34681a3f1d05bfb438f6e617d3ee,20914c76989e5eac4e2675de44f7275a,5,I definitely recommend,Good afternoon. I don't receive all the produc...,2024-08-28 00:00,2024-08-28 22:20
99085,7b85c6acaddb354af9aa840528838d9f,33a3edb84b9df4cb49546859b990ac6d,1,Unreliable company,Approved!,2024-03-21 00:00,2024-10-11 13:34
99095,dd8304dfce7be4ef6afdee5de01ebe8e,be1f74dd8c9f6ab6066aa38f00c87d99,5,perfect,"Original product, fast delivery time. Super sa...",2024-06-08 00:00,2024-06-08 16:41
99108,b01ab0ba0ad74ab16c994956e5cf20e6,f1bc2e9a22c1bd86a33e5f04595f1bdd,3,I highly recommend,"The product was not sent with an NF, there is ...",2024-07-04 00:00,2024-07-05 11:23


In [64]:
def fill_message(row):
    if pd.isna(row['Review_Comment_Message_En']):
        if row['Review_Score'] >= 4:
            return 'Positive review without comment'
        elif row['Review_Score'] == 3:
            return 'Neutral review without comment'
        else:
            return 'Negative review without comment'
    return row['Review_Comment_Message_En']

orders_reviews_no_emojis['Review_Comment_Message_En'] = orders_reviews_no_emojis.apply(fill_message, axis=1)

orders_reviews_no_emojis['Review_Comment_Title_En'] = orders_reviews_no_emojis['Review_Comment_Title_En'].fillna('No title')


In [65]:
orders_reviews_no_emojis.isnull().sum()

Review_ID                    0
Order_ID                     0
Review_Score                 0
Review_Comment_Title_En      0
Review_Comment_Message_En    0
Review_Creation_Date         0
Review_Answer_Timestamp      0
dtype: int64

In [66]:
orders_reviews_no_emojis_duplicates = orders_reviews_no_emojis[orders_reviews_no_emojis.duplicated(subset=['Review_ID'], keep=False)]
orders_reviews_no_emojis_duplicates

,Review_ID,Order_ID,Review_Score,Review_Comment_Title_En,Review_Comment_Message_En,Review_Creation_Date,Review_Answer_Timestamp
200,28642ce6250b94cc72bc85960aec6c62,e239d280236cdd3c40cb2c033f681d1c,5,No title,Positive review without comment,2024-03-25 00:00,2024-03-25 21:03
344,a0a641414ff718ca079b3967ef5c2495,169d7e0fd71d624d306f132acd791cbe,5,No title,Positive review without comment,2024-03-04 00:00,2024-03-06 20:12
346,f4d74b17cd63ee35efa82cd2567de911,f269e83a82f64baa3de97c2ebf3358f6,3,No title,"The packaging left a lot to be desired, the pr...",2024-01-12 00:00,2024-01-13 18:46
360,ecbaf1fce7d2c09bfab46f89065afeaf,2451b9756f310d4cff5c7987b393870d,5,No title,Positive review without comment,2023-07-27 00:00,2023-07-28 16:57
393,6b1de94de0f4bd84dfc4136818242faa,92acf87839903a94aeca0e5040d99acb,5,No title,Positive review without comment,2024-02-16 00:00,2024-02-19 19:04
...,...,...,...,...,...,...,...
99107,2c6c08892b83ba4c1be33037c2842294,42ae1967f68c90bb325783ac55d761ce,4,No title,Positive review without comment,2023-07-03 00:00,2023-07-05 19:06
99123,6ec93e77f444e0b1703740a69122e35d,e1fdc6e9d1ca132377e862593a7c0bd4,5,No title,Positive review without comment,2023-10-07 00:00,2023-10-07 19:47
99163,2afe63a67dfd99b3038f568fb47ee761,c5334d330e36d2a810a7a13c72e135ee,5,No title,Positive review without comment,2024-03-03 00:00,2024-03-04 22:56
99166,017808d29fd1f942d97e50184dfb4c13,b1461c8882153b5fe68307c46a506e39,5,No title,Positive review without comment,2024-03-02 00:00,2024-03-05 01:43


In [67]:
review_filtered = orders_reviews_no_emojis[orders_reviews_no_emojis['Review_ID'] == '44d1e9165ec54b1d89d33594856af859']
review_filtered

,Review_ID,Order_ID,Review_Score,Review_Comment_Title_En,Review_Comment_Message_En,Review_Creation_Date,Review_Answer_Timestamp
72812,44d1e9165ec54b1d89d33594856af859,a1d50a1883814a31c8629adf354c1013,4,No title,"I haven't installed it yet, I received it yest...",2023-05-24 00:00,2023-05-24 23:15
99177,44d1e9165ec54b1d89d33594856af859,a7dbcf5043158d6fa72859eead2f3d10,4,No title,Positive review without comment,2023-05-24 00:00,2023-05-24 23:15


In [68]:
orders_reviews_no_emojis = orders_reviews_no_emojis.drop_duplicates(subset=['Review_ID'], keep='first')

In [69]:
orders_reviews_no_emojis_duplicates = orders_reviews_no_emojis[orders_reviews_no_emojis.duplicated(subset=['Review_ID'], keep=False)]
orders_reviews_no_emojis_duplicates

,Review_ID,Order_ID,Review_Score,Review_Comment_Title_En,Review_Comment_Message_En,Review_Creation_Date,Review_Answer_Timestamp


### Descubrimientos clave:
- La columna `Review_Comment_Message_En` presentaba valores faltantes.  
- La columna `Review_Comment_Title_En` también contenía valores nulos.  
- Se detectaron posibles duplicados en `Review_ID`.  
- Existían reseñas asociadas a órdenes que no estaban en la tabla `orders`.  

### Transformaciones aplicadas:
- **Imputación de comentarios faltantes:**  
  En los casos donde `Review_Comment_Message_En` estaba vacío, se imputó un mensaje genérico dependiendo del puntaje de la reseña (`Review_Score`):  
  - `>= 4`: *"Positive review without comment"*  
  - `= 3`: *"Neutral review without comment"*  
  - `< 3`: *"Negative review without comment"*  

- **Imputación de títulos faltantes:**  
  Los valores nulos en `Review_Comment_Title_En` se reemplazaron con `"No title"`.  

- **Eliminación de duplicados:**  
  Se eliminaron reseñas duplicadas, conservando la primera ocurrencia por `Review_ID`.  

- **Filtrado de reseñas válidas:**  
  Solo se conservaron las reseñas asociadas a órdenes presentes en la tabla `orders`, garantizando la integridad referencial.  
